# Preface
I will be looking at some common digital marketing metrics by exploring the Google Analytics data of the Google Mercandise Store, an ecommerce site. I will primarily be using SQL and bq_helper to assess the data from BigQuery, along with some pandas and seaborn functionsfor additional processing and visualizations. 

In [1]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['train_v2.csv', 'test_v2.csv', 'sample_submission_v2.csv']


In [2]:
import bq_helper

# https://www.kaggle.com/sohier/introduction-to-the-bq-helper-package
bq_assistant = bq_helper.BigQueryHelper("bigquery-public-data", "google_analytics_sample")
print(bq_assistant.list_tables()[:5])
print(bq_assistant.list_tables()[-5:])
table_names = bq_assistant.list_tables()
# a table for each day : 2016-08-01 - 2017-08-01
# a year's worth of data

Using Kaggle's public dataset BigQuery integration.
['ga_sessions_20160801', 'ga_sessions_20160802', 'ga_sessions_20160803', 'ga_sessions_20160804', 'ga_sessions_20160805']
['ga_sessions_20170728', 'ga_sessions_20170729', 'ga_sessions_20170730', 'ga_sessions_20170731', 'ga_sessions_20170801']


# Look at available columns

In [3]:
bq_assistant.head("ga_sessions_20160801", num_rows=5)
# Many nested columns 
    # BiQuery allows access via "category_name.subcategory_name"
# What do rows represent?
    # Each row within a table corresponds to a session in Analytics 360.


,visitorId,visitNumber,visitId,visitStartTime,date,totals,trafficSource,device,geoNetwork,customDimensions,hits,fullVisitorId,userId,channelGrouping,socialEngagementType
0,None,1,1470117657,1470117657,20160801,"{'visits': 1, 'hits': 3, 'pageviews': 3, 'time...","{'referralPath': '/yt/about/', 'campaign': '(n...","{'browser': 'Internet Explorer', 'browserVersi...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 23, 'minu...",7194065619159478122,None,Social,Not Socially Engaged
1,None,151,1470083489,1470083489,20160801,"{'visits': 1, 'hits': 3, 'pageviews': 3, 'time...","{'referralPath': '/yt/about/', 'campaign': '(n...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 13, 'minu...",8159312408158297118,None,Social,Not Socially Engaged
2,None,1,1470052694,1470052694,20160801,"{'visits': 1, 'hits': 4, 'pageviews': 3, 'time...","{'referralPath': '/yt/about/', 'campaign': '(n...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Asia', 'subContinent': 'Southea...",[],"[{'hitNumber': 1, 'time': 0, 'hour': 4, 'minut...",9236304747882138291,None,Social,Not Socially Engaged
3,None,1,1470061879,1470061879,20160801,"{'visits': 1, 'hits': 4, 'pageviews': 4, 'time...","{'referralPath': '/yt/about/', 'campaign': '(n...","{'browser': 'Firefox', 'browserVersion': 'not ...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 7, 'minut...",1792676004815023069,None,Social,Not Socially Engaged
4,None,1,1470090830,1470090830,20160801,"{'visits': 1, 'hits': 4, 'pageviews': 2, 'time...","{'referralPath': '/yt/about/', 'campaign': '(n...","{'browser': 'Internet Explorer', 'browserVersi...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 15, 'minu...",7305625498291809599,None,Social,Not Socially Engaged


Description of columns available at https://support.google.com/analytics/answer/3437719?hl=en  

Notable columns:
1. fullVisitorId: str, unique visitor ID (hashed) 
2. visitID - int, identifier to session - only unique within user -- completely unique id is 1+2


In [4]:
last_schema = bq_assistant.table_schema("ga_sessions_20170801")
first_col_names = bq_assistant.table_schema("ga_sessions_20160801")['name']
last_col_names = bq_assistant.table_schema("ga_sessions_20170801")['name']
print("Number of columns in 2016:", len(first_col_names))
print("Number of columns in 2017:", len(last_col_names))

Number of columns in 2016: 336
Number of columns in 2017: 338


In [5]:
# print new columns 
[c for c in last_col_names if c not in first_col_names.tolist()]

['hits.product.productCouponCode', 'clientId']

In [6]:
def inspect(query, nrows=15, sample=False):
    """Display response from given query but don't save. 
    query: str, raw SQL query
    nrows: int, number of rows to display, default 15
    sample: bool, use df.sample instead of df.head, default False """
    response = bq_assistant.query_to_pandas_safe(query, max_gb_scanned=10)
    if sample:
        return response.sample(nrows)
    return response.head(nrows) 

def retrieve(query, nrows=10):
    """Save response from given query and print a preview. 
    query: str, raw SQL query
    nrows: int, number of rows to display"""
    response = bq_assistant.query_to_pandas_safe(query, max_gb_scanned=10)
    print(response.head(nrows))
    return response

## Check for duplicates 

In [7]:
query = """
SELECT
    COUNT(*)
FROM
    `bigquery-public-data.google_analytics_sample.ga_sessions_20160801`
"""
inspect(query)

,f0_
0,1711


In [8]:
query = """
SELECT COUNT(*)
FROM 
     (SELECT DISTINCT fullVisitorId, visitID
        FROM
    `bigquery-public-data.google_analytics_sample.ga_sessions_20160801`) s
"""
inspect(query)

,f0_
0,1711


In [9]:
# Total Sessions
query = """
SELECT
    COUNT(*)
FROM
    `bigquery-public-data.google_analytics_sample.ga_sessions_*`
"""
inspect(query)
# The wild card parses to a UNION ALL

,f0_
0,903653


In [10]:
# Unique sessions 
query = """
SELECT COUNT(*)
FROM 
     (SELECT DISTINCT fullVisitorId, visitID
        FROM
    `bigquery-public-data.google_analytics_sample.ga_sessions_*`) s
"""
inspect(query)

# A few(~900) sessions don't have unique ID's if looking across all tables

,f0_
0,902755


In [11]:
# Quick Check of alternative method Unique sessions 
query = """
SELECT COUNT(*)
FROM 
     (SELECT DISTINCT concat(fullVisitorId, cast(visitID as string))
        FROM
    `bigquery-public-data.google_analytics_sample.ga_sessions_*`) s
"""
inspect(query)

,f0_
0,902755


In [12]:
# Test alt way to count distinct 
table = table_names[0]
print(table)
date = table[-8:]
print(date)
query = f"""
SELECT {date} as table, 
COUNT(*) as total, 
count(DISTINCT CONCAT(fullVisitorId, CAST(visitID as string))) as unique
FROM 
    `bigquery-public-data.google_analytics_sample.{table}`
"""
inspect(query)

ga_sessions_20160801
20160801


,table,total,unique
0,20160801,1711,1711


In [13]:
# ' UNION '.join(['A', 'B', 'C'])

In [14]:
# Create a bunch of queries with a loop
# queries = []
# for table in table_names:
#     date = table[-8:]
#     query = f"""SELECT {date}, COUNT(*) as total, count(DISTINCT CONCAT(fullVisitorId, CAST(visitID as string))) as unique FROM `bigquery-public-data.google_analytics_sample.{table}`"""
#     queries.append(query)
# queries[:5]

In [15]:
# %%time # wall time 9 mins

# tables = []
# mismatches = []
# for query in queries:
#     table_info = bq_assistant.query_to_pandas_safe(query)
#     tables.append(table_info)
#     if table_info.loc[0, 'total'] != table_info.loc[0, 'unique']:
#         print(table_info['date'])
#         mismatches.append(table_info)

In [16]:
# # List of tables with record counts
# table_records = pd.concat(tables, axis=0)
# table_records

In [17]:
# Test unnesting- needed since it's a list within the column
unnested_query = """
SELECT
    fullVisitorId,
    visitId,
    visitNumber,
    hits.hitNumber AS hitNumber,
    hits.page.pagePath AS pagePath
FROM
    `bigquery-public-data.google_analytics_sample.ga_sessions_20160801`, UNNEST(hits) as hits
WHERE
    hits.type="PAGE"
ORDER BY
    fullVisitorId,
    visitId,
    visitNumber,
    hitNumber
"""
unnested_query_df = retrieve(unnested_query)
unnested_query_df



         fullVisitorId                        ...                                                                   pagePath
0   000722514342430295                        ...                                                                      /home
1   000722514342430295                        ...                                                   /google+redesign/apparel
2   000722514342430295                        ...                                                              /asearch.html
3   000722514342430295                        ...                                                              /asearch.html
4  0014659935183303341                        ...                                                                      /home
5  0015694432801235877                        ...                                                                      /home
6  0015694432801235877                        ...                                               /google+redesign/electronics


,fullVisitorId,visitId,visitNumber,hitNumber,pagePath
0,000722514342430295,1470093727,1,1,/home
1,000722514342430295,1470093727,1,3,/google+redesign/apparel
2,000722514342430295,1470093727,1,4,/asearch.html
3,000722514342430295,1470093727,1,5,/asearch.html
4,0014659935183303341,1470037282,1,1,/home
5,0015694432801235877,1470043732,1,1,/home
6,0015694432801235877,1470043732,1,2,/google+redesign/electronics
7,0015694432801235877,1470043732,1,3,/google+redesign/apparel/men++s/men++s+t+shirts
8,0015694432801235877,1470043732,1,4,/google+redesign/apparel/kid+s/kid+s+infant
9,0015694432801235877,1470043732,1,5,/google+redesign/apparel/kid+s/kid+s+infant/qu...


In [18]:
# How many unique visitors ?
query = """ SELECT COUNT (DISTINCT fullVisitorId) unique_visitors
            FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`            
            """
inspect(query)

,unique_visitors
0,714167


In [19]:
# How many unique customers ?
query = """ SELECT COUNT (DISTINCT fullVisitorId) unique_visitors
            FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`
            WHERE totals.transactions > 0 
            """
inspect(query)

,unique_visitors
0,10022


In [20]:
# Repeat customers and how many sessions with a purchase
query = """ SELECT fullVisitorId, COUNT(DISTINCT visitId) cnt_purchases
            FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`
            WHERE totals.transactions > 0 
            GROUP BY fullVisitorId
            HAVING COUNT(DISTINCT visitId) > 1
            """
        
repeat_customers = retrieve(query)


         fullVisitorId  cnt_purchases
0  4869304818683909994              2
1   773808986747306437              2
2   743123551680199202              2
3   832152661091318994              2
4  6095240958469403590              2
5  4988517937139937145              5
6  8205269849536680674              2
7  7594157089401877562              2
8  8039691484832064792              2
9  5341271361784611942              5


In [21]:
print(repeat_customers.shape)
repeat_customers['cnt_purchases'].sum()


(970, 2)


2499

# Inspiration Questions:

1. [What is the total number of transactions generated per device browser in July 2017?](#insp1)
2. [The real bounce rate is defined as the percentage of visits with a single pageview. What was the real bounce rate per traffic source?](#insp2)
3. [What was the average number of product pageviews for users who made a purchase in July 2017?](#insp3)
4. [What was the average number of product pageviews for users who did not make a purchase in July 2017?](#insp4)
5. [What was the average total transactions per user that made a purchase in July 2017?](#insp5)
6. [What is the average amount of money spent per session in July 2017?](#insp6)
7. [What is the sequence of pages viewed?](#insp7)

<a id='insp1'> </a>
What is the total number of transactions generated per device browser in July 2017?

In [ ]:
# group by device browser from july tables 
query = f"""
SELECT device.browser, 
    COUNT(totals.totalTransactionRevenue) AS cnt_revenue,
    COUNT(totals.transactions) AS cnt_transactions, 
    COUNT(*) AS cnt_all_rows, --sessions with browser
    COUNT(totals.transactions) / COUNT(*) AS sess_conversion_rate,
    sum(totals.totalTransactionRevenue) AS sum_revenue, 
    Sum(totals.transactions) AS  sum_transactions,
    sum(totals.totalTransactionRevenue) / Sum(totals.transactions) AS revenue_per_transaction
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_201707*` 
 GROUP BY device.browser
 ORDER BY sum_transactions DESC
"""

transactions_by_browser = retrieve(query)

In [ ]:
transactions_by_browser.head(10)
# Most transactions are done through Chrome. It also has the highest session-to-transaction conversion rate and revenue per transaction.

<a id='insp2'> </a>
The real bounce rate is defined as the percentage of visits with a single pageview. What was the real bounce rate per traffic source?

In [ ]:
# "Real" bounce rate -- go by definition and filter where total.pageviews = 1 and divide by total visits
query = """
SELECT t.source,
    t.total_visits,
    b.bounce_visits,
    100 * b.bounce_visits / t.total_visits AS bounce_rate
FROM 
(SELECT trafficSource.source, COUNT(visitId) AS total_visits 
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_201707*`
GROUP BY trafficSource.source) t

JOIN (SELECT trafficSource.source, COUNT(visitId) bounce_visits 
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_201707*`
WHERE totals.pageviews = 1
GROUP BY trafficSource.source) b
ON t.source = b.source
ORDER by total_visits DESC

"""
inspect(query)

# Slightly different results than using totals.bounces column - more bounces visits but total visits are the same

In [ ]:
# GA bounce rate 
query = """
SELECT trafficSource.source, 
    COUNT(visitId) AS total_visits,
    COUNT(totals.bounces) AS bounce_visits,
    100 * COUNT(totals.bounces) / COUNT(visitId) AS bounce_rate
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_201707*`
GROUP BY trafficSource.source
ORDER BY total_visits DESC
"""

inspect(query)

In [ ]:
query = """
SELECT DISTINCT totals.pageviews AS pageview_values
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_201707*`
WHERE totals.bounces= 1
"""
inspect(query)

In [ ]:
query = """
SELECT DISTINCT totals.bounces AS bounce_values
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_201707*`
WHERE totals.pageviews= 1
"""
inspect(query)

# total.bounces column - for convenience but is sometimes null for a session with 1 pageview -- explains the extra bounces 

In [ ]:
# Cofirm same results as other kernel
howto_query = """SELECT
source,
total_visits,
total_no_of_bounces,
( ( total_no_of_bounces / total_visits ) * 100 ) AS bounce_rate
FROM (
SELECT
trafficSource.source AS source,
COUNT ( trafficSource.source ) AS total_visits,
SUM ( totals.bounces ) AS total_no_of_bounces
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE
_TABLE_SUFFIX BETWEEN '20170701' AND '20170731'
GROUP BY
source )
ORDER BY
total_visits DESC;
        """
inspect(howto_query)

<a id='insp3'> </a>
What was the average number of product pageviews for users who made a purchase in July 2017?


In [ ]:
# get users who made a purchase in July 2017
query = """
SELECT DISTINCT fullVisitorId 
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_201707*`
WHERE totals.transactions > 0
"""
inspect(query)

In [ ]:
# more info on users who made a purchase in July 2017
query = """
SELECT 
    COUNT(fullVisitorId) AS total_visitors, 
    COUNT(DISTINCT fullVisitorId) AS unique_visitors, 
    SUM(totals.transactions) AS sum_transactions,
    SUM(totals.totalTransactionRevenue) AS sum_revenue,
    AVG(totals.pageviews) as avg_pageviews -- in sessions where purchase was made

FROM `bigquery-public-data.google_analytics_sample.ga_sessions_201707*`
WHERE totals.transactions > 0
"""
inspect(query)

In [ ]:
# calculate average among those users who made a purchase in July 2017
query = """
SELECT AVG(totals.pageviews) as avg_pageviews -- includes other sessions by user in which they did not make a purchase
 FROM `bigquery-public-data.google_analytics_sample.ga_sessions_201707*`
WHERE fullVisitorId IN (SELECT DISTINCT fullVisitorId 
                         FROM `bigquery-public-data.google_analytics_sample.ga_sessions_201707*`
                        WHERE totals.transactions > 0)
"""
inspect(query)

<a id='insp4'> </a>

What was the average number of product pageviews for users who did not make a purchase in July 2017?  
<t>Slight differences in the average for both questions 3 and 4 depending on interpretation of question but overall trend is that users who make a purchase have more than 3 times the page views as users who do not make a purchase.

In [ ]:
query = """
SELECT 
    COUNT(fullVisitorId) AS total_visitors, 
    COUNT(DISTINCT fullVisitorId) AS unique_visitors, 
    SUM(totals.transactions) AS sum_transactions,
    SUM(totals.totalTransactionRevenue) AS sum_revenue,
    AVG(totals.pageviews) as avg_pageviews -- in sessions where purchase were not made

FROM `bigquery-public-data.google_analytics_sample.ga_sessions_201707*`
WHERE totals.transactions IS NULL
"""
inspect(query)

In [ ]:
# calculate average among those users who did not made a purchase in July 2017
query = """
SELECT AVG(totals.pageviews) as avg_pageviews -- includes other sessions by user in which they did make a purchase
 FROM `bigquery-public-data.google_analytics_sample.ga_sessions_201707*`
WHERE fullVisitorId IN (SELECT DISTINCT fullVisitorId 
                         FROM `bigquery-public-data.google_analytics_sample.ga_sessions_201707*`
                        WHERE totals.transactions IS NULL)
"""
inspect(query)

<a id='insp5'> </a>

What was the average total transactions per user that made a purchase in July 2017?



In [ ]:
query = """
SELECT 
    COUNT(fullVisitorId) AS total_visitors, 
    COUNT(DISTINCT fullVisitorId) AS unique_visitors, 
    SUM(totals.transactions) AS sum_transactions,
    AVG(totals.totalTransactionRevenue) AS sum_revenue,
    AVG(totals.transactions) as avg_transactions, -- in sessions where purchase were  made
    SUM(totals.transactions) / COUNT(fullVisitorId) as alt_avg_transactions
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_201707*`
WHERE totals.transactions > 0
"""
inspect(query)

In [ ]:
# calculate average among those users who made a purchase in July 2017
query = """
SELECT AVG(totals.transactions) as avg_transactions,  -- includes other sessions by user in which they did not make a purchase
      SUM(totals.transactions) / COUNT(fullVisitorId) as alt_avg_transactions -- AVG function ignores NULLs by default
 FROM `bigquery-public-data.google_analytics_sample.ga_sessions_201707*`
WHERE fullVisitorId IN (SELECT DISTINCT fullVisitorId 
                         FROM `bigquery-public-data.google_analytics_sample.ga_sessions_201707*`
                        WHERE totals.transactions > 0)
"""
inspect(query)

<a id='insp6'> </a>
What is the average amount of money spent per session in July 2017?



In [ ]:
query = """
SELECT 
    AVG(totals.totalTransactionRevenue) as avg_money_spent_per_purchase,
    SUM(totals.totalTransactionRevenue) / COUNT(*) as avg_money_spent
 FROM `bigquery-public-data.google_analytics_sample.ga_sessions_201707*`
"""
inspect(query)

In [ ]:
 
howto_query6 = """SELECT
( SUM(total_transactionrevenue_per_user) / SUM(total_visits_per_user) ) AS
avg_revenue_by_user_per_visit
FROM (
SELECT
fullVisitorId,
SUM( totals.visits ) AS total_visits_per_user,
SUM( totals.transactionRevenue ) AS total_transactionrevenue_per_user
FROM
`bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE
_TABLE_SUFFIX BETWEEN '20170701' AND '20170731'
AND
totals.visits > 0
AND totals.transactions >= 1
AND totals.transactionRevenue IS NOT NULL
GROUP BY
fullVisitorId );
"""
inspect(howto_query6)
# Why group by VisitorId (ie user) when it asks for money spent per session? 

In [ ]:
query = """
SELECT 
   COUNT(totals.visits) as visits,
   COUNT(*) - COUNT(totals.visits) as sessions_not_visits
 FROM `bigquery-public-data.google_analytics_sample.ga_sessions_201707*`
--WHERE totals.visits  1
"""
inspect(query) 
# so all sessions are visits, as it should be 

<a id='insp7'> </a>
What is the sequence of pages viewed?

In [ ]:
howto_query7 = """SELECT
fullVisitorId,
visitId,
visitNumber,
hits.hitNumber AS hitNumber,
hits.page.pagePath AS pagePath
FROM
`bigquery-public-data.google_analytics_sample.ga_sessions_*`,
UNNEST(hits) as hits
WHERE
_TABLE_SUFFIX BETWEEN '20170701' AND '20170731'
AND
hits.type="PAGE"
ORDER BY
fullVisitorId,
visitId,
visitNumber,
hitNumber;
        """
view_seqs = retrieve(howto_query7)

In [ ]:
print(view_seqs.shape)
view_seqs.head()

In [ ]:
# hitNumber has the order within the session
# view_seqs.groupby('fullVisitorId')['visitNumber'].count() # find a sample customer with more than 1 visit
view_seqs[view_seqs['fullVisitorId'] == '0000436683523507380']

In [ ]:
# apply list to each group - a way to aggregate the different records into the same row
# nice to see but not recommended to store a list in a dataframe
view_seqs.groupby(['fullVisitorId', 'visitId']).agg({'hitNumber': list, 'pagePath': list}).head(5)

In [ ]:
# Get first page from each session - could be nice to use for first touch attribution  
# view_seqs.groupby(['fullVisitorId', 'visitId']).head(1)
# Top 10 landing pages from sessions - homepage 50% of time
view_seqs.groupby(['fullVisitorId', 'visitId']).head(1)['pagePath'].value_counts(normalize=True).head(10)

# Exploring other columns
Sticking with July 2017
1. trafficSource.source
2. trafficSource.medium
3. device.browser
4. device.deviceCategory
5. totals.sessionQualityDim
6. totals.timeOnScreen
<p hidden> 7. GROUP BY Source SELECT sum(Revenue) - c * Count(impressions) </p>

### TrafficSource

In [ ]:
# expand traffic source 
query = """SELECT
fullVisitorId,
visitId,
visitNumber,
hits.hitNumber AS hitNumber,
trafficSource.*
FROM
`bigquery-public-data.google_analytics_sample.ga_sessions_*`,
UNNEST(hits) as hits
WHERE
_TABLE_SUFFIX BETWEEN '20170701' AND '20170731'
;
        """
inspect(query)


In [ ]:
# What are possible sources? in July 2017 value_counts
query = """SELECT
trafficSource.source, COUNT(*) as cnt
FROM
`bigquery-public-data.google_analytics_sample.ga_sessions_201707*`
GROUP BY trafficSource.source
ORDER BY cnt DESC
        """
traffic_sources = retrieve(query)

In [ ]:
print(traffic_sources.shape)
print("Too many to plot, even if establishing a threshold at like 100 for July. Need a good way of combining sources")
traffic_sources.head(15)

In [ ]:
# confirm (direct) is the source of the (none)'s
query = """SELECT
 trafficSource.source, trafficSource.medium, count(*) cnt
FROM
`bigquery-public-data.google_analytics_sample.ga_sessions_201707*`
WHERE -- trafficSource.source IN ('google', '(direct)') AND 
 trafficSource.medium = '(none)'
GROUP BY 1,2
ORDER BY cnt DESC

        """
inspect(query)

### What are the possible values for various columns of interest?

In [ ]:
# What are possible media? in 2017 value_counts
query = """SELECT
trafficSource.medium, COUNT(*) as cnt
FROM
`bigquery-public-data.google_analytics_sample.ga_sessions_2017*`
GROUP BY trafficSource.medium
ORDER BY cnt DESC
        """
inspect(query)
# cpc: cost per click - use when has good offer with high conversion rate using ads adapted to that specific offer
# cpm: cost per thousand impression - may save money if good CTR, can buy traffic from premium spots
# cpv = cost per unique visitor

In [ ]:
# What are possible devices? value_counts
query = """SELECT
device.browser, COUNT(*) as cnt
FROM
`bigquery-public-data.google_analytics_sample.ga_sessions_201707*`
GROUP BY device.browser
ORDER BY cnt DESC
        """
inspect(query)


In [ ]:
# What are possible device categories? value_counts
query = """SELECT
device.deviceCategory, COUNT(*) as cnt
FROM
`bigquery-public-data.google_analytics_sample.ga_sessions_201707*`
GROUP BY device.deviceCategory
ORDER BY cnt DESC
        """
inspect(query)

In [ ]:
# What are possible browsers? value_counts -- counts unique users -  but session is more relevant
query = """
SELECT
 deviceCategory, 
 COUNT(*) as cnt
FROM
    (SELECT 
      fullVisitorId,
      visitId,
      MAX(device.deviceCategory) as deviceCategory
    FROM
     `bigquery-public-data.google_analytics_sample.ga_sessions_201707*`
    GROUP BY fullVisitorId, visitId
    ) s
GROUP BY deviceCategory
ORDER BY cnt DESC
        """
inspect(query)

**totals.sessionQualityDim**: An estimate of how close a particular session was to transacting, ranging from 1 to 100, calculated for each session. A value closer to 1 indicates a low session quality, or far from transacting, while a value closer to 100 indicates a high session quality, or very close to transacting. A value of 0 indicates that Session Quality is not calculated for the selected time range.

<p> How well does it predict actual conversion/transacting? Come back to this. </p> 

In [ ]:
query = """
SELECT  
 totals.sessionQualityDim,
 COUNT(*) cnt
FROM 
`bigquery-public-data.google_analytics_sample.ga_sessions_201707*`
GROUP BY totals.sessionQualityDim
ORDER by cnt DESC

"""
sess_quality_vc = retrieve(query)

In [ ]:
import seaborn as sns
sns.set()
sns.scatterplot(x='sessionQualityDim', y='cnt', data=sess_quality_vc[sess_quality_vc['sessionQualityDim'] > 2])
# exclude the two 

In [ ]:
sess_quality_vc.sort_values('sessionQualityDim').tail(10)

In [ ]:
query = """
SELECT  
 totals.timeOnSite,
 COUNT(*) cnt
FROM 
`bigquery-public-data.google_analytics_sample.ga_sessions_2017*`
GROUP BY totals.timeOnSite
ORDER by cnt DESC

"""
time_on_site = retrieve(query) 
#  totals.timeOnScreen - not a valid column
# timeOnSite: Total time of the session expressed in seconds.

In [ ]:
time_on_site['timeOnSite'].describe()

## Having found interesting columns to explore, let's see how they might affect conversion rate.


In [ ]:
# Base conversion rate in July
query = f"""
SELECT 
    COUNT(totals.totalTransactionRevenue) AS cnt_revenue,
    COUNT(totals.transactions) AS cnt_transactions, 
    COUNT(*) AS cnt_all_rows, --sessions with browser
    COUNT(totals.transactions) / COUNT(*) AS sess_conversion_rate,
    sum(totals.totalTransactionRevenue) AS sum_revenue, 
    Sum(totals.transactions) AS  sum_transactions,
    sum(totals.totalTransactionRevenue) / Sum(totals.transactions) AS revenue_per_transaction
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_201707*` 
"""

base_CR = retrieve(query)

In [ ]:
base_CR

### Base conversion rate by month 

In [ ]:
# Base conversion rate by month 
query = f"""
SELECT 
    substr(date, 1, 6) as ym,
    COUNT(totals.totalTransactionRevenue) AS cnt_revenue,
    COUNT(totals.transactions) AS cnt_transactions, 
    COUNT(*) AS cnt_all_rows, --sessions with browser
    COUNT(totals.transactions) / COUNT(*) AS sess_conversion_rate,
    sum(totals.totalTransactionRevenue) AS sum_revenue, 
    Sum(totals.transactions) AS  sum_transactions,
    sum(totals.totalTransactionRevenue) / Sum(totals.transactions) AS revenue_per_transaction
FROM `bigquery-public-data.google_analytics_sample.ga_sessions*` 
GROUP BY ym
ORDER BY ym
"""

monthly_base_CR = retrieve(query)

In [ ]:
monthly_base_CR
# Not all transactionshave revenue? 

In [ ]:
monthly_base_CR['ym'] = pd.to_datetime(monthly_base_CR['ym'], format='%Y%m')

In [ ]:
sns.lineplot(x='ym', y='sess_conversion_rate', data=monthly_base_CR)

In [ ]:
sns.lineplot(x='ym', y='cnt_all_rows', data=monthly_base_CR)
# the big CR dip in Nov is due more to higher traffic than a drop in transactions

### Base conversion rate by month and device category

In [ ]:
# Base conversion rate by month and device category
query = """
SELECT 
    substr(date, 1, 6) as ym,
    device.deviceCategory,
    COUNT(totals.totalTransactionRevenue) AS cnt_revenue,
    COUNT(totals.transactions) AS cnt_transactions, 
    COUNT(*) AS cnt_all_rows, --sessions with browser
    COUNT(totals.transactions) / COUNT(*) AS sess_conversion_rate,
    sum(totals.totalTransactionRevenue) AS sum_revenue, 
    Sum(totals.transactions) AS  sum_transactions,
    sum(totals.totalTransactionRevenue) / Sum(totals.transactions) AS revenue_per_transaction
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`
GROUP BY  device.deviceCategory, ym
ORDER BY device.deviceCategory, ym
"""
monthly_device_CR = retrieve(query)
monthly_device_CR.head()

In [ ]:
import matplotlib.pyplot as plt
def plot_metric_by_month(vc_df, metric, group):
    '''Convert date from string and plot metric by group
    vc_df: df, SQL value counts output grouped by month and group
    metric: str, column name for metric of interest
    group: str, column name of group'''
    df = vc_df.copy()
    df['ym'] = pd.to_datetime( df['ym'], format='%Y%m')
    sns.lineplot(x='ym', y=metric, hue=group, data=df, marker='o')
    plt.title(f'Monthly {metric}')
    plt.legend(loc="upper left", bbox_to_anchor=(1,1))

In [ ]:
plot_metric_by_month(monthly_device_CR, 'sess_conversion_rate', 'deviceCategory')
# much higher CR when visited via desktop

In [ ]:
plot_metric_by_month(monthly_device_CR, 'cnt_transactions', 'deviceCategory')
# raw count looks similar in shape for desktop and mobile, so likely consistent total traffic but changing number of transactions

### Base conversion rate by month and traffic source

In [ ]:
# Base conversion rate by month and traffic source
query = """
SELECT 
    substr(date, 1, 6) as ym,
    trafficSource.medium, 
    COUNT(totals.totalTransactionRevenue) AS cnt_revenue,
    COUNT(totals.transactions) AS cnt_transactions, 
    COUNT(*) AS cnt_all_rows, --sessions with browser
    COUNT(totals.transactions) / COUNT(*) AS sess_conversion_rate,
    sum(totals.totalTransactionRevenue) AS sum_revenue, 
    Sum(totals.transactions) AS  sum_transactions,
    sum(totals.totalTransactionRevenue) / Sum(totals.transactions) AS revenue_per_transaction
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`
GROUP BY trafficSource.medium, ym
ORDER BY trafficSource.medium, ym
"""
monthly_medium_CR = retrieve(query)

In [ ]:
plot_metric_by_month(monthly_medium_CR, 'sess_conversion_rate', 'medium')
# direct ads (cpc and cpm) > organic > referral/affiliate
# medium can have a pretty strong effect on CR - diverging from the overall average of 1.5% 

In [ ]:
plot_metric_by_month(monthly_medium_CR, 'cnt_transactions', 'medium')
# organic has a low CR but it makes up for a good chunk of the transactions
# Need to look further into (none) --> direct traffic - so people who want to buy stuff often visit directly 
# - bookmarks? returning customers?

### Base conversion rate by month and browser 

In [ ]:
# Base conversion rate by month and browser 
query = """
SELECT 
    substr(date, 1, 6) as ym,
    device.browser,
    COUNT(totals.totalTransactionRevenue) AS cnt_revenue,
    COUNT(totals.transactions) AS cnt_transactions, 
    COUNT(*) AS cnt_all_rows, --sessions with browser
    COUNT(totals.transactions) / COUNT(*) AS sess_conversion_rate,
    sum(totals.totalTransactionRevenue) AS sum_revenue, 
    Sum(totals.transactions) AS  sum_transactions,
    sum(totals.totalTransactionRevenue) / Sum(totals.transactions) AS revenue_per_transaction
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`
GROUP BY  device.browser, ym
ORDER BY  device.browser, ym
"""
monthly_browser_CR = retrieve(query)

In [ ]:
mask = (monthly_browser_CR['sess_conversion_rate'] > 0) | (monthly_browser_CR['cnt_all_rows'] > 1000) # doesn't catch every month if a browser doesn't always make it
print(len(monthly_browser_CR[mask]))
monthly_browser_CR[mask].sample(10)

In [ ]:
plot_metric_by_month(monthly_browser_CR[mask], 'sess_conversion_rate', 'browser')
# Chrome dominates both raw count and CR
# Silk has that one blip of relatively high CR

In [ ]:
plot_metric_by_month(monthly_browser_CR[mask], 'cnt_transactions', 'browser')


In [ ]:
plot_metric_by_month(monthly_browser_CR[mask & (monthly_browser_CR['browser'] != 'Chrome')], 'cnt_transactions', 'browser')
# Safari being a distant second but another sizeable gap from the rest

### Base conversion rate by month and source

In [ ]:

# Base conversion rate by month and source
query = """
SELECT 
    substr(date, 1, 6) as ym,
    trafficSource.source,
    COUNT(totals.totalTransactionRevenue) AS cnt_revenue,
    COUNT(totals.transactions) AS cnt_transactions, 
    COUNT(*) AS cnt_all_rows, --sessions with browser
    COUNT(totals.transactions) / COUNT(*) AS sess_conversion_rate,
    sum(totals.totalTransactionRevenue) AS sum_revenue, 
    Sum(totals.transactions) AS  sum_transactions,
    sum(totals.totalTransactionRevenue) / Sum(totals.transactions) AS revenue_per_transaction
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`
GROUP BY trafficSource.source, ym
ORDER BY  trafficSource.source, ym
"""
monthly_source_CR = retrieve(query)

In [ ]:
monthly_source_CR.head(10)

In [ ]:
monthly_source_CR.shape

In [ ]:
mask = (monthly_source_CR['cnt_all_rows'] > 500) 
plot_metric_by_month(monthly_source_CR[mask], 'sess_conversion_rate', 'source')
# dominated by direct or google
# not sure what dfa is

In [ ]:
mask = (monthly_source_CR['cnt_all_rows'] > 500) 
plot_metric_by_month(monthly_source_CR[mask], 'sess_conversion_rate', 'source')

We've found that conversion rate and the number of transactions have associations with the traffic source as well the device and browser used for the visit. Direct visits and paid ads appear to be positively correlated with conversion ratios. Chrome users make up an overwhelming majority of conversions. 



# To do's
<p> This is a rich data set, so a lot remains to be explored. This includes looking at different campaigns, closer looks into specific audiences, other definitions of conversion, other metrics like ROI and ROAS. An interesting feature is the totals.sessionQualityDim, which is GA's estimate on how close a session came to converting. There are other ones like totals.timeOnScreen, which may be more a vanity metric but could be insightful on the user experience. </p>

Misc Queries



In [22]:
# looking into a particuar visitor
query = """SELECT date, visitId, channelGrouping, visitNumber, trafficSource.medium, 
            totals.*
            FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`
            WHERE  fullVisitorId =  '7813149961404844386'
            ORDER BY date 
            
            """
q = retrieve(query)



       date        ...          sessionQualityDim
0  20161117        ...                       None
1  20161118        ...                       None
2  20161118        ...                       None
3  20161122        ...                       None
4  20161122        ...                       None
5  20161202        ...                       None
6  20161206        ...                       None
7  20161206        ...                       None
8  20161206        ...                       None
9  20161206        ...                       None

[10 rows x 18 columns]


In [ ]:
q.head()

In [ ]:
# channelgrouping - ga channels
query = """SELECT channelGrouping, COUNT(*) cnt
            FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`
            GROUP BY channelGrouping
            """
inspect(query)

In [ ]:
query = """SELECT channelGrouping, COUNT(DISTINCT trafficSource.medium) cnt
            FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`
            GROUP BY channelGrouping
            """
inspect(query)

In [ ]:
# verify conditions match up with definition  https://support.google.com/analytics/answer/3297892
query = """SELECT channelGrouping, array_agg(DISTINCT trafficSource.medium) cnt
            FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`
            GROUP BY channelGrouping
            """
inspect(query)

In [ ]:
# above by with counts within each group
query = """SELECT channelGrouping,trafficSource.medium, count(*) cnt
            FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`
            GROUP BY channelGrouping,trafficSource.medium
            """
inspect(query)

In [ ]:
# group by medium
query = """SELECT trafficSource.medium,  channelGrouping, count(*) cnt
            FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`
            GROUP BY trafficSource.medium, channelGrouping
            ORDER BY trafficSource.medium
            """
inspect(query)
# so (none) can have labeled in channelGroupig even when 

In [ ]:
# what do these look like? 
query = """SELECT * 
            FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`
            WHERE trafficSource.medium = '(none)'
                AND channelGrouping = 'Referral'
            
            """
inspect(query)

In [ ]:
query = """
SELECT 
    substr(date, 1, 6) as ym,
    channelGrouping,
    COUNT(totals.totalTransactionRevenue) AS cnt_revenue,
    COUNT(totals.transactions) AS cnt_transactions, 
    COUNT(*) AS cnt_all_rows, --sessions with browser
    COUNT(totals.transactions) / COUNT(*) AS sess_conversion_rate,
    sum(totals.totalTransactionRevenue) AS sum_revenue, 
    Sum(totals.transactions) AS  sum_transactions,
    sum(totals.totalTransactionRevenue) / Sum(totals.transactions) AS revenue_per_transaction
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`
GROUP BY  channelGrouping, ym
ORDER BY channelGrouping, ym
"""
monthly_channel_CR = retrieve(query)
monthly_channel_CR.head()

In [ ]:
monthly_channel_CR.sum()

In [ ]:
plot_metric_by_month(monthly_channel_CR, 'sess_conversion_rate', 'channelGrouping')

In [ ]:
plot_metric_by_month(monthly_channel_CR, 'cnt_transactions', 'channelGrouping')
# Why does this look so different than raw source medium
    # because (none) can be assigned to various channelGroupings and other caveats
# https://support.google.com/analytics/answer/3297892

In [ ]:
# Base conversion rate by month and traffic source
query = """
SELECT 
    substr(date, 1, 6) as ym,
    trafficSource.medium, 
    COUNT(totals.totalTransactionRevenue) AS cnt_revenue,
    COUNT(totals.transactions) AS cnt_transactions, 
    COUNT(*) AS cnt_all_rows, --sessions with browser
    COUNT(totals.transactions) / COUNT(*) AS sess_conversion_rate,
    sum(totals.totalTransactionRevenue) AS sum_revenue, 
    Sum(totals.transactions) AS  sum_transactions,
    sum(totals.totalTransactionRevenue) / Sum(totals.transactions) AS revenue_per_transaction
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`
GROUP BY trafficSource.medium, ym
ORDER BY trafficSource.medium, ym
"""
monthly_medium_CR = retrieve(query)

In [ ]:
monthly_medium_CR.sum()
# revenue per transaction and sess_conversion_rate changed -- rest stayed the same

In [ ]:
plot_metric_by_month(monthly_medium_CR, 'sess_conversion_rate', 'medium')
# direct ads (cpc and cpm) > organic > referral/affiliate
# medium can have a pretty strong effect on CR - diverging from the overall average of 1.5% 

In [ ]:
plot_metric_by_month(monthly_medium_CR, 'cnt_transactions', 'medium')
# organic has a low CR but it makes up for a good chunk of the transactions
# Need to look further into (none) --> direct traffic - so people who want to buy stuff often visit directly 
# - bookmarks? returning customers?

Resources and Acknowledgements:
* Table schema https://support.google.com/analytics/answer/3437719?hl=en
* BigQueryHelper https://www.kaggle.com/sohier/introduction-to-the-bq-helper-package
* BigQuery tricks: https://www.kaggle.com/vikramtiwari/bigquery-in-depth-using-google-analytics-data
* BQ Standard SQL - Unnest: https://cloud.google.com/bigquery/docs/reference/standard-sql/query-syntax#unnest